In [12]:
import os
import json

import numpy as np
import matplotlib as plt
from rich import print as pprint
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import auc

from truthful_counterfactuals.utils import EXPERIMENTS_PATH
from truthful_counterfactuals.metrics import threshold_error_reduction
from truthful_counterfactuals.visualization import plot_threshold_error_reductions
from truthful_counterfactuals.utils import latex_table, render_latex, latex_table_element_mean

PATH = os.getcwd()
RESULTS_PATH = os.path.join(EXPERIMENTS_PATH, 'results')
RESULTS_FILE = 'results.json'
MODEL_FILE = 'model.ckpt'

NUM_BINS = 50

In [22]:
result_map = {
    # == GAT RESULTS ==
    'random': {
        'model': 'GATv2',
        'method': 'Random',
        'paths': [
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__rand', 'logp_gat_1'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__rand', 'logp_gat_2'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__rand', 'logp_gat_3'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__rand', 'logp_gat_4'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__rand', 'logp_gat_5'),
        ]  
    },
    'gat_ens_cal': {
        'model': 'GATv2',
        'method': 'ENS(3)',
        'paths': [
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gat_cal_1'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gat_cal_2'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gat_cal_3'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gat_cal_4'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gat_cal_5'),
        ]
    },
    'gat_mve_cal': {
        'model': 'GATv2',
        'method': 'MVE',
        'paths': [
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gat_cal_1'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gat_cal_2'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gat_cal_3'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gat_cal_4'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gat_cal_5'),
        ]
    },
    'gat_swag_cal': {
        'model': 'GATv2',
        'method': 'SWAG',
        'paths': [
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__swag', 'logp_gat_cal_1'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__swag', 'logp_gat_cal_2'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__swag', 'logp_gat_cal_3'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__swag', 'logp_gat_cal_4'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__swag', 'logp_gat_cal_5'),
        ]
    },
    'gat_ens_mve_cal': {
        'model': 'GATv2',
        'method': 'ENS(3)+MVE',
        'paths': [
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens_mve', 'logp_gat_cal_1'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens_mve', 'logp_gat_cal_2'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens_mve', 'logp_gat_cal_3'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens_mve', 'logp_gat_cal_4'),
            os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens_mve', 'logp_gat_cal_5'),
        ]
    },
    # == GIN RESULTS ==
    # 'gin_ens': {
    #     'model': 'GIN',
    #     'method': 'ENS(3)',
    #     'paths': [
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_1'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_2'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_3'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_4'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_5'),
    #     ]
    # },
    # 'gin_ens_cal': {
    #     'model': 'GIN',
    #     'method': 'ENS(3)+Cal.',
    #     'paths': [
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_cal_1'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_cal_2'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_cal_3'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_cal_4'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__ens', 'logp_gin_cal_5'),
    #     ]
    # },
    # 'gin_mve': {
    #     'model': 'GIN',
    #     'method': 'MVE',
    #     'paths': [
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_1'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_2'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_3'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_4'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_5'),
    #     ]
    # },
    # 'gin_mve_cal': {
    #     'model': 'GIN',
    #     'method': 'MVE+Cal.',
    #     'paths': [
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_cal_1'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_cal_2'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_cal_3'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_cal_4'),
    #         os.path.join(RESULTS_PATH, 'quantify_uncertainty__mve', 'logp_gin_cal_5'),
    #     ]
    # }
}

def get_value(value: float | list):
    if isinstance(value, list):
        return value[0]
    return value

In [23]:
print('processing the results...')
for key, data in result_map.items():
    
    data['mae_values'] = []
    data['r2_values'] = []
    data['corr_values'] = []
    data['auc_mean_values'] = []
    data['auc_max_values'] = []
    
    print(f' * processing {key}')
    for path in data['paths']:
        
        results_path = os.path.join(path, RESULTS_FILE)
        with open(results_path, mode='r') as file:
            content = file.read()
            results = json.loads(content)
            
        out_true = [get_value(result['graph_labels']) for result in results]
        out_pred = [get_value(result['prediction']) for result in results]
            
        # calculating the prediction performance metrics (MAE & R2)
        mae_value = mean_absolute_error(out_true, out_pred)
        r2_value = r2_score(out_true, out_pred)
        data['mae_values'].append(mae_value)
        data['r2_values'].append(r2_value)
        
        # as a setup we then have to calculate the error between the prediction and the true value
        # and also get the uncertainty values from the results directly
        for result in results:
            result['error'] = abs(get_value(result['graph_labels']) - get_value(result['prediction']))
        
        errors = np.array([get_value(result['error']) for result in results])
        uncertainties = np.array([get_value(result['uncertainty']) for result in results])
        
        # then we can calculate the correlation between the error and the uncertainty
        corr_value = np.corrcoef(errors, uncertainties)[0, 1]
        data['corr_values'].append(corr_value)
        
        # finally we can use the error and uncertainty values to calculate the EUT-AUC
        ths, rds = threshold_error_reduction(uncertainties, errors, error_func=np.mean, num_bins=NUM_BINS)
        auc_mean_value = auc(ths, rds)
        data['auc_mean_values'].append(auc_mean_value)
        
        ths, rds =  threshold_error_reduction(uncertainties, errors, error_func=np.max, num_bins=NUM_BINS)
        auc_max_value = auc(ths, rds)
        data['auc_max_values'].append(auc_max_value)
        
    pprint(data)

processing the results...
 * processing random


{
    'model': 'GATv2',
    'method': 'Random',
    'paths': [
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__rand/logp_gat_1',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__rand/logp_gat_2',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__rand/logp_gat_3',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__rand/logp_gat_4',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__rand/logp_gat_5'
    ],
    'mae_values': [
        0.22994149329451039,
        0.1089681889225306,
        0.08857477922730474,
        0.11079045765633827,
        0.12031867420725034
    ],
    'r2_values': [
        0.9887749430365377,
        0.9974299425222598,
        0.9980715212330038,
        0.9966892944712803,
        0.9967681788341608
    ],
    'corr_values': [
        0.0016261160963285252,
        -0.023172416664321387,
        -0.01008786938613326,
        0.026003956611392213,
        0.0507511158479331
    ],
    'auc_mean_values': [
        -0.004712764081840015,
        -0.03707289757429034,
        -0.018966939771370667,
        0.0404276060222675,
        0.0764914281510512
    ],
    'auc_max_values': [
        0.08249025948047004,
        0.29350865147941746,
        0.008437282290240265,
        0.03233650163185148,
        0.0711583503784218
    ]
}

 * processing gat_ens_cal


{
    'model': 'GATv2',
    'method': 'ENS(3)',
    'paths': [
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens/logp_gat_cal_1',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens/logp_gat_cal_2',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens/logp_gat_cal_3',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens/logp_gat_cal_4',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens/logp_gat_cal_5'
    ],
    'mae_values': [
        0.08742114232441978,
        0.11953952734760452,
        0.0883939092523431,
        0.08104246322779877,
        0.08855863073952992
    ],
    'r2_values': [
        0.9979234860578983,
        0.9956697449259213,
        0.9979140670594936,
        0.9983034673213381,
        0.997770944880367
    ],
    'corr_values': [
        0.5262409022134348,
        0.4704025302204361,
        0.42395457531777536,
        0.48313613465517397,
        0.37866147265241423
    ],
    'auc_mean_values': [
        0.20625108172180076,
        0.22117205426908731,
        0.23742383378170734,
        0.16951118494528053,
        0.23843571028059324
    ],
    'auc_max_values': [
        0.7641825517064224,
        0.25930838063959066,
        0.5651657254811459,
        0.6759620461197409,
        0.26260912830571764
    ]
}

 * processing gat_mve_cal


{
    'model': 'GATv2',
    'method': 'MVE',
    'paths': [
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__mve/logp_gat_cal_1',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__mve/logp_gat_cal_2',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__mve/logp_gat_cal_3',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__mve/logp_gat_cal_4',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__mve/logp_gat_cal_5'
    ],
    'mae_values': [
        0.10569132759889963,
        0.17751715126876344,
        0.7834454482871659,
        0.6704280332547481,
        0.08461341529568683
    ],
    'r2_values': [
        0.9954790659501883,
        0.9917230476159811,
        0.8524983537123189,
        0.862153488160444,
        0.9901898573556899
    ],
    'corr_values': [
        0.5644940330262079,
        0.5384174780054235,
        0.625395441410859,
        0.7788230813176648,
        0.8543946692488021
    ],
    'auc_mean_values': [
        0.288635953301025,
        0.1717721815311604,
        0.32801212434086124,
        0.326034895872122,
        0.3874595951752551
    ],
    'auc_max_values': [
        0.8419775543957702,
        0.5549297636576251,
        0.5159484173687021,
        0.8291144334824516,
        0.9517949114672105
    ]
}

 * processing gat_swag_cal


{
    'model': 'GATv2',
    'method': 'SWAG',
    'paths': [
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__swag/logp_gat_cal_1',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__swag/logp_gat_cal_2',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__swag/logp_gat_cal_3',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__swag/logp_gat_cal_4',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__swag/logp_gat_cal_5'
    ],
    'mae_values': [
        0.1277603386813929,
        0.1023196485464945,
        0.104554221539711,
        0.09836335557599389,
        0.11255503535419704
    ],
    'r2_values': [
        0.9968474697749296,
        0.99819868074612,
        0.9980806861797826,
        0.9977403937814984,
        0.9973282814514963
    ],
    'corr_values': [
        0.45314439733576733,
        0.23332749277372047,
        0.29618572995815945,
        0.5758905867554756,
        0.26856482332544235
    ],
    'auc_mean_values': [
        0.20559902725854826,
        0.18696925674473464,
        0.14348720827918646,
        0.24963516078072426,
        0.12844316580734771
    ],
    'auc_max_values': [
        0.5954075354506829,
        0.36918367737860736,
        0.5068574024892106,
        0.5584982952802903,
        0.11385894277138504
    ]
}

 * processing gat_ens_mve_cal


{
    'model': 'GATv2',
    'method': 'ENS(3)+MVE',
    'paths': [
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens_mve/logp_gat_cal_1',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens_mve/logp_gat_cal_2',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens_mve/logp_gat_cal_3',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens_mve/logp_gat_cal_4',
        '/media/ssd/Programming/truthful_counterfactuals/truthful_counterfactuals/experiments/results/quantify_unce
rtainty__ens_mve/logp_gat_cal_5'
    ],
    'mae_values': [
        0.3769345907475507,
        0.658783883544674,
        0.23661861355476754,
        0.0959489247408293,
        0.12931695785838265
    ],
    'r2_values': [
        0.9477884599682938,
        0.8986170623774389,
        0.9850227749297439,
        0.9970886955924144,
        0.9938885795500448
    ],
    'corr_values': [
        0.7422183528592704,
        0.7319195907217888,
        0.6701917297958896,
        0.6843554910369802,
        0.6205588050369099
    ],
    'auc_mean_values': [
        0.2982683770388865,
        0.33708709060388997,
        0.23450344836640874,
        0.2808718329275624,
        0.32732301544277
    ],
    'auc_max_values': [
        0.7097953494469227,
        0.7691603990759498,
        0.7762844812248094,
        0.7951953267424908,
        0.816830692209336
    ]
}

In [24]:
print('summarizing the results...')

rows = []
for key, data in result_map.items():
    row = [
        data['model'],
        data['method'],
        data['mae_values'],
        data['r2_values'],
        data['corr_values'],
        data['auc_mean_values'],
        data['auc_max_values'],
    ]
    rows.append(row)

_, content = latex_table(
    column_names=['Model', 'Method', 'MAE', r'$R^2$', r'$\rho$', r'$\text{UER-AUC}_{\text{mean}}$', r'$\text{UER-AUC}_{\text{max}}$'],
    rows=rows,
)

tex_path = os.path.join(PATH, 'table_1.tex')
with open(tex_path, mode='w') as file:
    file.write(content)
    
pdf_path = os.path.join(PATH, 'table_1.pdf')
render_latex({'content': content}, pdf_path)



summarizing the results...


In [25]:
from IPython.display import display, IFrame, display_pdf

display_pdf(pdf_path)